In [1]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain


In [2]:
import prompts as p
from langchain.prompts import PromptTemplate


PRODUCTS = '../../datasets/amazon_walmart.tsv'

PRODUCT_MAPPING = '../../datasets/WalAmazonDataset_sampled.csv'

SAVED_PROMPTS_PATH = '../../evals/saved_prompts_WA.csv'

# Load data
df_products = pd.read_csv(PRODUCTS, sep='\t')
df_mapping = pd.read_csv(PRODUCT_MAPPING)


df_products.fillna('', inplace=True)
df_mapping.fillna('', inplace=True)

prompt_name = 'prompt1_en_short_temp_0_gpt3.5-0301'
prompt = p.prompt1_en_short

MODEL = "gpt-3.5-turbo-0301"

MAX_TOKENS = 15


In [78]:
# generate column with prompt_name if not exists
if 'prompt_name' not in df_mapping.columns:
    df_mapping[prompt_name] = ""

In [79]:
df_mapping.head(30)

,Unnamed: 0,id1,id2,label,prompt1_en_short_temp_0_gpt3.5-0301
0,0,7784,23390,True,
1,1,15123,23390,True,
2,2,21113,23390,True,
3,3,7384,22973,True,
4,4,11344,22973,True,
5,5,15336,22973,True,
6,6,8176,24537,True,
7,7,14118,24537,True,
8,8,20398,23769,True,
9,9,11027,23769,True,


In [80]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI


In [81]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [82]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [83]:
def cut_length(string ,length = 250):
    return string[0:length]

In [84]:

# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none"))
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none"))



# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none")) + "\nBrand:" + str(df["brand_left"][1] or "none") 
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none")) + "\nBrand:" + str(df["brand_right"][1] or "none") 

In [85]:
saved_prompts_df = pd.read_csv(SAVED_PROMPTS_PATH)

def saveResponse(prompt, response, isMatch, index):
    # create new column if not exists 
    if "prompt_" + prompt_name not in saved_prompts_df.columns:
        saved_prompts_df["prompt_" + prompt_name] = ""
        saved_prompts_df["response_" + prompt_name] = ""
        saved_prompts_df["isMatch_" + prompt_name] = ""


    
    # save response prompt and isMatch to df at index
    saved_prompts_df.at[index, "prompt_" + prompt_name] = prompt
    saved_prompts_df.at[index, "response_" + prompt_name] = response
    saved_prompts_df.at[index, "isMatch_" + prompt_name] = isMatch
    
    

In [86]:
# print(df_products)
df_products
print(str(df_products["longdescr"][0]))


the pocket size koss 3 band equalizer delivers high fidelity performance and output normally reserved for more expensive home systems with a 10db boost or 10db cut range of level it features a 3 band equalizer that allows for convenient and individual bass midrange and treble adjustment power output is greater than 20mw per channel providing clean and undistorted output into your favorite stereophones ergonomically designed for easy handling a rotary volume control and on off switch are placed for convenient usage 


In [87]:
def evaluate(df, prompt, temp=0, start = 0):
    # start = len(df) - start - 1
    if (MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
        print("Using OpenAI")
    else:
        llm = ChatOpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
    for i in range(start, len(df)):
        print(i)
        # Creating Prompt and sending request and printing informations
        
        product1 = "Title: " + cut_length(str(df_products["title"][df["id1"][i]-1] or "none")) + "\nDescription: " + cut_length(str(df_products["longdescr"][df["id1"][i]-1] or "none")) 
        product2 = "Title: " + cut_length(str(df_products["title"][df["id2"][i]-1] or "none")) + "\nDescription: " + cut_length(str(df_products["longdescr"][df["id2"][i]-1] or "none")) 

        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        
        try:
            response = chain.run(p1=product1, p2=product2, callbacks=[handler])        
        except Exception as e:
            print(e)
            i -= 1
            continue

        formatted_prompt = chain.prompt.format_prompt(p1=product1, p2=product2).text
        # if(i < 5):
        print(formatted_prompt)
        print("__\n__")
        print(response)
        print("__\n__")
        
        #Processing Response    
        isMatch = False
        if ("True" in response or "true" in response or "yes" in response):
            isMatch = True
        elif ("False" in response or "false" in response or "no" in response):
            isMatch = False
        
        df[prompt_name][i] = isMatch


        saveResponse(formatted_prompt, response, isMatch, i)

In [88]:
evaluate(df_mapping, prompt,temp=0, start=0)


0


Are the following two products the same?
Product 1: Title: optoma bl fp230c replacement projector lamp bulb with housing high quality replacement lamp
Description: this shopforbattery part number sfp 213 123271 is the premium projector lamp that replaced the optoma bl fp230c this projector lamp is a brand new lamp with new housing and tested to be 100 oem compatible it is different from other sellers that only 
Product 2: Title: optoma bl fp230c replacement projector lamp for optoma ep749 projector
Description: none

Answer with 'True' or 'False'.
__
__
True
__
__
1


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_72710/372998102.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: electrified bl fp230c sp 85r01gc01 replacement lamp with housing for optoma projectors
Description: brand new replacement projection lamp with brand new housing for optoma projectors 150 day electrified warranty
Product 2: Title: optoma bl fp230c replacement projector lamp for optoma ep749 projector
Description: none

Answer with 'True' or 'False'.
__
__
False
__
__
2


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_72710/372998102.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


...

Are the following two products the same?
Product 1: Title: new lowrance 000 10236 001 elite tm 5 dsi fishfinder base lra10236001
Description: lowrance 000 10236 001 elite tm 5 dsi fishfinder base lowrance downscan imaging tm displays clear photos of views of underwater underneath the boat clearly see boundary layer between lower cooler water amp upper warmer water where fish are located di
Product 2: Title: lowrance elite 5 dsi fishfinder base
Description: lowrance elite 5 dsi fishfinder base detailed photo like viewing easy to use manual fine tuning option 5 256 color tft display 480 x 480 pixels 500w rms power trackback feature built in precision gps antenna

Answer with 'True' or 'False'.
__
__
True.
__
__
12


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_72710/372998102.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


Are the following two products the same?
Product 1: Title: lowrance 000 10236 001 elite tm 5 dsi fishfinder base
Description: lowrance 000 10236 001 elite tm 5 dsi fishfinder base lowrance downscan imaging tm displays clear photos of views of underwater underneath the boat clearly see boundary layer between lower cooler water amp upper warmer water where fish are located di
Product 2: Title: lowrance elite 5 dsi fishfinder base
Description: lowrance elite 5 dsi fishfinder base detailed photo like viewing easy to use manual fine tuning option 5 256 color tft display 480 x 480 pixels 500w rms power trackback feature built in precision gps antenna

Answer with 'True' or 'False'.
__
__
True.
__
__
13


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_72710/372998102.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


KeyboardInterrupt: 

In [ ]:
df_mapping


,Unnamed: 0,id1,id2,label,prompt1_en_short_temp_0_gpt3.5-0301
0,0,7784,23390,True,True
1,1,15123,23390,True,False
2,2,21113,23390,True,True
3,3,7384,22973,True,True
4,4,11344,22973,True,False
...,...,...,...,...,...
400,400,23912,9467,False,False
401,401,23912,9468,False,False
402,402,23912,9469,False,False
403,403,23912,9470,False,False


In [ ]:
# saved_prompts_df

In [ ]:
# write df to csv
df_mapping.to_csv(PRODUCT_MAPPING, index=False)
saved_prompts_df.to_csv(SAVED_PROMPTS_PATH, index=False)
